In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [107]:
import os
import glob
from matplotlib import colors
from keras.utils import to_categorical

In [85]:
def resize_img(im,grayscale,desired_size = 512 ):
#   im = Image.open(im_pth)
#   old_size = im.size  # old_size[0] is in (width, height) format
#   im = cv2.imread(im_pth)
  old_size = im.shape[0:2]
  if grayscale:
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
  old_size = im.shape[:2] # old_size is in (height, width) format
  ratio = float(desired_size)/max(old_size)
  new_size = tuple([int(x*ratio) for x in old_size])

  # new_size should be in (width, height) format
  im = cv2.resize(im, (new_size[1], new_size[0])) 

  delta_w = desired_size - new_size[1]
  delta_h = desired_size - new_size[0]
  top, bottom = delta_h//2, delta_h-(delta_h//2)
  left, right = delta_w//2, delta_w-(delta_w//2)

  color = [0, 0, 0]
  new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
      value=color)
  new_im = cv2.cvtColor(new_im,cv2.COLOR_BGR2RGB)
  return new_im

In [86]:
def prepare_image_label(img_path,lbl_vsl_path,lbl_od_path,lbl_msk_path,plot_before):
    # 1) read images
    img = cv2.imread(img_path) #BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #RGB
    lbl_vsl = cv2.imread(lbl_vsl_path,0)
    lbl_od = cv2.imread(lbl_od_path,0)
    lbl_msk = cv2.imread(lbl_msk_path,0)
    if plot_before:
        fig,ax = plt.subplots(nrows=1,ncols=4,figsize=(20,5))
        ax[0].imshow(img)
        ax[1].imshow(lbl_vsl, cmap='Greys_r')
        ax[2].imshow(lbl_od, cmap='Greys_r')
        ax[3].imshow(lbl_msk, cmap='Greys_r')
        ax[0].set_title('original image')
        ax[1].set_title('vessel')
        ax[2].set_title('optic disc')
        ax[3].set_title('vitreous body')
        plt.show()
        
    # 2) assign optic disc to 1
    lbl_od[lbl_od==lbl_msk]=255
    lbl_od[lbl_od==0]=1 # optic disc
    lbl_od[lbl_od==255]=0
    # 3) assign vessel to 2
    lbl_vsl
    lbl_vsl[lbl_vsl == 255] = 2 # vessel
    lbl_vsl[lbl_od==1] = 1
    # 4) assign floor-eye layer to 3
    lbl_msk[lbl_msk==255]=3 # eye-floor layer
    lbl_msk[lbl_od==1]=1
    lbl_msk[lbl_vsl==2]=2
    # 5) bitwise_or operation for all label image
    lbl_cmb = cv2.bitwise_or(lbl_od,lbl_vsl)
    lbl_cmb = cv2.bitwise_or(lbl_cmb,lbl_msk)
    lbl_cmb_cvt = cv2.cvtColor(lbl_cmb,cv2.COLOR_GRAY2RGB)
    return img,lbl_cmb_cvt
    

In [121]:
def get_sub_volume(image, label, 
                   orig_x = 1024, orig_y = 1024, orig_z = 3, 
                   output_x = 512, output_y = 512, output_z = 3,
                   num_classes = 4, max_tries = 1000, 
                   background_threshold=0.8):
    """
    Extract random sub-volume from original images.

    Args:
        image (np.array): original image, 
            of shape (orig_x, orig_y, orig_z, num_channels)
        label (np.array): original label. 
            labels coded using discrete values rather than
            a separate dimension, 
            so this is of shape (orig_x, orig_y, orig_z)
        orig_x (int): x_dim of input image
        orig_y (int): y_dim of input image
        orig_z (int): z_dim of input image
        output_x (int): desired x_dim of output
        output_y (int): desired y_dim of output
        output_z (int): desired z_dim of output
        num_classes (int): number of class labels
        max_tries (int): maximum trials to do when sampling
        background_threshold (float): limit on the fraction 
            of the sample which can be the background

    returns:
        X (np.array): sample of original image of dimension 
            (num_channels, output_x, output_y, output_z)
        y (np.array): labels which correspond to X, of dimension 
            (num_classes, output_x, output_y, output_z)
    """
    # Initialize features and labels with `None`
    X = None
    y = None

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    tries = 0
    
    while tries < max_tries:
        # randomly sample sub-volume by sampling the corner voxel
        # hint: make sure to leave enough room for the output dimensions!
        start_x = np.random.randint(orig_x - output_x + 1)
        start_y = np.random.randint(orig_y - output_y + 1)
#         start_z = np.random.randint(orig_z - output_z + 1)

        # extract relevant area of label
        y = label[start_x: start_x + output_x,
                  start_y: start_y + output_y,
                  :]
#                   start_z: start_z + output_z]
        
        # One-hot encode the categories.
        # This adds a 4th dimension, 'num_classes'
        # (output_x, output_y, output_z, num_classes)
#         print(y.shape)
        yi=y.copy()
        y = to_categorical(y, num_classes=num_classes)
#         print(y.shape)
#         print(y.shape,np.sum(y[:,:,:,0]),len(y))
        # compute the background ratio
        bgrd_ratio = np.sum(y[:,:,:,0])/(output_x*output_y*output_z)

        # increment tries counter
        tries += 1

        # if background ratio is below the desired threshold,
        # use that sub-volume.
        # otherwise continue the loop and try another random sub-volume
#         print(bgrd_ratio)
        if bgrd_ratio < background_threshold:
#             print(bgrd_ratio)

            # make copy of the sub-volume
            X = np.copy(image[start_x: start_x + output_x,
                              start_y: start_y + output_y,
                              :])
#                               start_z: start_z + output_z, :])
            
            # change dimension of X
            # from (x_dim, y_dim, z_dim, num_channels)
            # to (num_channels, x_dim, y_dim, z_dim)
#             X = np.moveaxis(X,-1,0)

            # change dimension of y
            # from (x_dim, y_dim, z_dim, num_classes)
            # to (num_classes, x_dim, y_dim, z_dim)
            y = np.moveaxis(y,-1,0)

            ### END CODE HERE ###
            
            # take a subset of y that excludes the background class
            # in the 'num_classes' dimension
            y = y[1:, :, :]
#             yx2=yx[]
#             print(start_x,start_y)
            return X, y,yi,start_x,start_y,bgrd_ratio

    # if we've tried max_tries number of samples
    # Give up in order to avoid looping forever.
    print(f"Tried {tries} times to find a sub-volume. Giving up...")

In [80]:
img_file.split('.')[0]+'.png'

'STAR 05_ODC.png'

In [138]:
main_dir = '/Volumes/GoogleDrive/My Drive/CMKL/datasets/IOSTAR/IOSTAR Vessel Segmentation Dataset'
img_folder = 'image'#'image_preprocessed','image'
img_out = 'image_label_png_patch'
if img_out not in os.listdir(main_dir):
    os.mkdir(os.path.join(main_dir,img_out))
count_img = 1
for img_file in os.listdir(os.path.join(main_dir,img_folder)):
    if img_file[0:4] != 'STAR':
        continue
    img_name = img_file.split('_')[0]
    print(img_name)
#     img_name = 'STAR 45'
    img_path = glob.glob(os.path.join(main_dir,img_folder,img_name+'*'))[0]
    lbl_vsl_path = glob.glob(os.path.join(main_dir,'GT',img_name+'*'))[0]
    lbl_od_path = glob.glob(os.path.join(main_dir,'mask_OD',img_name+'*'))[0]
    lbl_msk_path = glob.glob(os.path.join(main_dir,'mask',img_name+'*'))[0]
    plot_before = 0
    img,lbl_cmb_all = prepare_image_label(img_path,lbl_vsl_path,lbl_od_path,lbl_msk_path,plot_before)
    imgx = resize_img(img,False,248)
    lbl = resize_img(lbl_cmb_all,False,248)
    ##
    orig = 1024
    output = 248
    count_files=1
#     coord = []
    if count_img >=25:
        folder_name = 'val'
    else:
        folder_name = 'train'
    
    if folder_name+'_imgs' not in os.listdir(os.path.join(main_dir,img_out)):
        os.mkdir(os.path.join(main_dir,img_out,folder_name+'_imgs'))
        os.mkdir(os.path.join(main_dir,img_out,folder_name+'_imgs',folder_name))
        os.mkdir(os.path.join(main_dir,img_out,folder_name+'_masks'))
        os.mkdir(os.path.join(main_dir,img_out,folder_name+'_masks',folder_name))

        
#     print('hey1')
    while count_files <= 300:
        
        X, y,yi,start_x,start_y,bgrd_ratio = get_sub_volume(img, lbl_cmb_all,orig_x = orig, orig_y = orig, 
                   output_x = output, output_y = output)
        coord.append((start_x,start_y))
        
#         print('hey')
        X=cv2.cvtColor(X,cv2.COLOR_RGB2BGR)
        yi=cv2.cvtColor(yi,cv2.COLOR_RGB2BGR)
        
        file_out = img_file.split('.')[0]+'_x_'+str(start_x)+'_y_'+str(start_y)+'.png'
        cv2.imwrite(os.path.join(main_dir,img_out,folder_name+'_imgs',folder_name,file_out),X)
        cv2.imwrite(os.path.join(main_dir,img_out,folder_name+'_masks',folder_name,file_out),yi)
        count_files+=1
        
        
#         break
#     break
    count_img+=1
    ##
    
    
#     if count_img >=28:
#         folder_name = 'val'
#     elif count_img<24:
#         folder_name = 'train'
#     else:
#         folder_name = 'test'
#     if folder_name+'_imgs' not in os.listdir(main_dir):
#         os.mkdir(os.path.join(main_dir,img_out,folder_name+'_imgs'))
#         os.mkdir(os.path.join(main_dir,img_out,folder_name+'_imgs',folder_name))
#         os.mkdir(os.path.join(main_dir,img_out,folder_name+'_masks'))
#         os.mkdir(os.path.join(main_dir,img_out,folder_name+'_masks',folder_name))

#     file_out = img_file.split('.')[0]+'.png'
#     cv2.imwrite(os.path.join(main_dir,img_out,folder_name+'_imgs',folder_name,file_out),imgx)
#     cv2.imwrite(os.path.join(main_dir,img_out,folder_name+'_masks',folder_name,file_out),lbl)
#     count_img+=1
#     break      

STAR 05
STAR 13
STAR 17
STAR 02
STAR 01
STAR 03
STAR 06
STAR 08
STAR 16
STAR 10
STAR 09
STAR 15
STAR 30
STAR 28
STAR 31
STAR 26
STAR 21
STAR 24
STAR 20
STAR 37
STAR 34
STAR 36
STAR 43
STAR 40
STAR 48
STAR 44
STAR 39
STAR 38
STAR 45
STAR 32


In [130]:
os.listdir(main_dir)

['AV_GT',
 'GT',
 'image',
 'mask',
 'mask_OD',
 'patches',
 'config.json',
 'image_preprocessed',
 'config_processed.json',
 'patches_preprocessed_new',
 'config_processed_new.json',
 'history.pkl',
 'image_label_png',
 'image_label_png_patch',
 'Icon\r']

In [125]:
np.unique(yi)

array([0, 2, 3], dtype=uint8)